In [1]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


# Inicializa la autenticación y la inicialización de Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

In [2]:
# Definir la región de interés
# roi = ee.FeatureCollection('projects/facub-gee/assets/pruebaRF')

coords = [[-60.61950458631911,-31.856976338766977],
[-60.51942600355544,-31.856976338766977],
[-60.51942600355544,-31.78273238873983],
[-60.61950458631911,-31.78273238873983],
[-60.61950458631911,-31.856976338766977]]

# Crear la geometría del polígono
roi = ee.Geometry.Polygon([coords])
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
# Añadir Google Satellite como mapa base
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

In [9]:


# Definir las bandas para predecir la biomasa
bands = ['ARVI', 'B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'CH', 'DEM', 'EVI', 'GCI', 'MNDWI', 'NDBI', 'NDVI', 'SAVI', 'Slope', 'VH', 'VV', 'label']

# Parámetros de visualización para imágenes multiespectrales en falso color nir-swir1-swir2
visFalseColor = {
    'min': 0,
    'max': 3000,
    'bands': ['B8', 'B11', 'B12']
}

# Parámetros de visualización para la biomasa
visBiomass = {'min': 0, 'max': 100, 'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']}

# Parámetros de visualización para Sentinel-1
visParamsS1 = {
    'min': -25,
    'max': 0,
    'bands': ['VV', 'VH']
}

# Definir el año de análisis
year = 2020

# Definir la fecha de inicio y fin para filtrar la colección
start = f'{year}-01-01'
end = f'{year}-12-31'

# Filtrar la colección Sentinel-2
s2Col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(roi) \
    .filterDate(start, end)

# Filtrar la colección Sentinel-2 cloud score
cloudCol = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
    .filterBounds(roi) \
    .filterDate(start, end)

# Combinar la colección, máscara de nubes y hacer un composite mediano
def mask_clouds(image):
    cloud_score = ee.Image(cloudCol.filterMetadata('system:index', 'equals', image.get('system:index')).first())
    mask = cloud_score.select('probability').lt(10)
    return image.updateMask(mask)

image = s2Col.map(mask_clouds).median().toFloat().clip(roi)

# Calcular NDVI, MNDWI, NDBI, EVI, SAVI, GCI y ARVI
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
mndwi = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')
evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': image.select('B8'),
        'RED': image.select('B4'),
        'BLUE': image.select('B2')
    }).rename('EVI')
savi = image.expression(
    '((NIR - RED) / (NIR + RED + L)) * (1 + L)', {
        'NIR': image.select('B8'),
        'RED': image.select('B4'),
        'L': 0.5
    }).rename('SAVI')
gci = image.expression(
    '(NIR / GREEN) - 1', {
        'NIR': image.select('B8'),
        'GREEN': image.select('B3')
    }).rename('GCI')
arvi = image.expression(
    '(NIR - (RED - gamma * (RED - BLUE))) / (NIR + (RED - gamma * (RED - BLUE)))', {
        'NIR': image.select('B8'),
        'RED': image.select('B4'),
        'BLUE': image.select('B2'),
        'gamma': 1
    }).rename('ARVI')

# Añadir bandas NDVI, MNDWI, NDBI, EVI, SAVI, GCI y ARVI a la imagen
image = image.addBands(ndvi).addBands(mndwi).addBands(ndbi).addBands(evi).addBands(savi).addBands(gci).addBands(arvi)

# Definir la función de máscara de borde
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

def filter_iqr(image_collection, band):
    # Calcular percentiles
    percentiles = image_collection.reduce(ee.Reducer.percentile([25, 75]))
    p25 = percentiles.select(band + '_p25')
    p75 = percentiles.select(band + '_p75')
    # Calcular IQR
    iqr = p75.subtract(p25)
    # Filtrar valores fuera del rango IQR
    filtered = image_collection.map(
        lambda img: img.updateMask(
            img.select(band).gt(p25.subtract(iqr)).And(img.select(band).lt(p75.add(iqr)))
        )
    )
    return filtered

# Filtrar y procesar la colección Sentinel-1 para VV
img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(roi)
    .filterDate('2019-01-01', end)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(mask_edge)
)

# Filtrar y procesar la colección Sentinel-1 para VH
img_vh = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(roi)
    .filterDate('2019-01-01', end)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VH')
    .map(mask_edge)
)

# Aplicar filtro IQR a las colecciones VV y VH
img_vv_filtered = filter_iqr(img_vv, 'VV')
img_vh_filtered = filter_iqr(img_vh, 'VH')

# Promedio de imágenes VV y VH después de filtrar IQR
s1_vv = img_vv_filtered.median().rename('VV')
s1_vh = img_vh_filtered.median().rename('VH')

# Combinar las bandas VV y VH en una sola imagen
s1_median = ee.Image.cat([s1_vv, s1_vh]).clip(roi)

image = image.addBands(s1_median)

# Integrar la banda "label" de Dynamic World
dwCol = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate(start, end)

dwLabel = dwCol.select('label').median().clip(roi)

# Añadir la banda "label" de Dynamic World a la imagen
image = image.addBands(dwLabel.rename('label'))

# Cargar la imagen de altura de dosel
canopy_height = ee.Image("users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1").clip(roi)

# Añadir la banda de altura de dosel a la imagen
image = image.addBands(canopy_height.rename('CH'))



# Cargar la imagen de elevación desde ALOS AW3D30
elevation = ee.Image('JAXA/ALOS/AW3D30/V2_2').select('AVE_DSM').clip(roi)
# Añadir la banda de elevación a la imagen
image = image.addBands(elevation.rename('DEM'))

# Calcular la pendiente
terrain = ee.Terrain.products(elevation)
slope = terrain.select('slope')

# Añadir la banda de pendiente a la imagen
image = image.addBands(slope.rename('Slope'))

# Seleccionar solo las bandas especificadas
image = image.select(bands)

# Mostrar la imagen Sentinel-1 VV y VH en el mapa
Map = geemap.Map()
Map.addLayer(s1_median, visParamsS1, 'Sentinel-1 VV & VH', False)
Map.addLayer(image, visParamsS1, 'image', False)
Map.centerObject(roi, 12)

In [10]:
feature_names = ['ARVI', 'B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'CH', 'DEM', 'EVI', 'GCI', 'MNDWI', 'NDBI', 'NDVI', 'SAVI', 'Slope', 'VH', 'VV', 'label']
label = "agbd"

user_id = 'users/facuboladerasgee'
# specify asset id where to save trees
# be sure to change  to your ee user name
asset_id = user_id + "/Rf_Gee"
asset_id

# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)

In [21]:
# Reducir la imagen clasificada a su valor máximo en la región de interés
max_value = classified.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=roi,
    scale=30,
    maxPixels=1e9
)

# Obtener el valor máximo de la reducción
max_agbd = max_value.get('classification').getInfo()
print(f"Max value of the classified image: {max_agbd}")

Max value of the classified image: 54.255305009999994


In [13]:
# Centrar el mapa en la región de interés
Map = geemap.Map(center=[0, 0], zoom=2)
Map.centerObject(roi)

visPredictedBiomass = {
    'min': 0,
    'max': 60,
    'palette': ['lightyellow', 'lightgreen', 'green', 'darkgreen']
}

Map = geemap.Map(zoom=11)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

Map.addLayer(
    classified,
    visPredictedBiomass,
    "classification",)
Map.add_colorbar(visPredictedBiomass, label="Predicted Biomass")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…